# スタッキング
・GBDT 2~3個：決定木の深さが「浅い」「普通」「深い」モデル

・Random Forest 1~2個：決定木の深さが「浅い」「深い」モデル

・Neural Net 1~2個：層の数が「少ない」「多い」モデル

・Linier 1個

In [1]:
import os
os.chdir('../../')

In [2]:
import numpy as np
import pandas as pd
from scr.util import *
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from scr.models.gbdt import Model1_CatBoost_1, Model1_CatBoost_2, Model1_CatBoost_3, Model1_XGBoost_1, Model1_XGBoost_2, Model1_XGBoost_3, Model1_LightGBM_1, Model1_LightGBM_2, Model1_LightGBM_3
from scr.models.random_forest import Model1_RandomForest_1, Model1_RandomForest_2, Model1_RandomForest_3
from scr.models.nn import Model1_NN_1, Model1_NN_2, Model1_NN_all_1, Model1_NN_all_2, Model1_TabNet_1, Model1_TabNet_2
from scr.models.linear import Model1_Logistic_1, Model1_Logistic_2, Model1_Logistic_3, Model1_Logistic_4, Model2_Logistic

In [26]:
path_gb = 'data/sampling/under_sampling/mean_gb'
path_nn = 'data/sampling/under_sampling/mean_nn'

files_gb = os.listdir(path_gb)
files_nn = os.listdir(path_nn)

trains_gb = []
for file in files_gb:
    file_path = os.path.join(path_gb, file)
    df = pd.read_csv(file_path)
    trains_gb.append(df)

trains_nn = []
for file in files_nn:
    file_path = os.path.join(path_nn, file)
    df = pd.read_csv(file_path)
    trains_nn.append(df)

In [27]:
train_gb = trains_gb[0]
test_gb = pd.read_csv('data/feature_engineered/null_representative/test_null_mean.csv')

train_nn = trains_nn[0]
test_nn = pd.read_csv('data/feature_engineered/null_representative/test_null_mean_scaled.csv')

combination_columns = train_gb.columns[44:110]
y = train_gb['ProdTaken']

In [28]:
# 特徴量が多いので、落とす特徴量を選択
drop_gb = [
    'EconomicSegment',
    'ContractRate_FM',
    'ContractRate_G1',
    'ContractRate_G2',
    'ContractRate_G3',
    'ContractRate_G4',
    'ContractRate_G5',
    'ContractRate_G6'
]

## --------------------------------------------------------------------------------------------
## Label Encoding
train_gb = mapping_columns_if_exist(train_gb)
test_gb = mapping_columns_if_exist(test_gb)

def handle_unknown_label(train_series, test_series):
    unique_labels = train_series.unique()
    label_map = {label: idx for idx, label in enumerate(unique_labels)}
    train_encoded = train_series.map(label_map)
    test_encoded = test_series.map(lambda x: label_map.get(x, -1))
    return train_encoded, test_encoded

for col in combination_columns:
    train_gb[col], test_gb[col] = handle_unknown_label(train_gb[col], test_gb[col])
## --------------------------------------------------------------------------------------------

X_gb = train_gb.drop(columns=drop_gb, axis=1)
y_gb = train_gb['ProdTaken']

test_feature = X_gb.columns.drop('ProdTaken')
test_gb = test_gb[test_feature]

tmp = X_gb.groupby(by=['AgeGroup', 'ProductPitched'], as_index=False)['ProdTaken'].mean()
tmp = tmp.rename(columns={'ProdTaken': 'ContractRate_G4'})
test_gb = test_gb.merge(tmp, on=['AgeGroup', 'ProductPitched'], how='left')

In [30]:
drop_nn = [
    'EconomicSegment',
    'ContractRate_FM',
    'ContractRate_G1',
    'ContractRate_G2',
    'ContractRate_G3',
    'ContractRate_G4',
    'ContractRate_G5',
    'ContractRate_G6'
]

X_nn = train_nn.drop(columns=drop_nn, axis=1)
X_nn = X_nn.drop(columns=['ProdTaken'])
y_nn = train_nn['ProdTaken']

test_nn = test_nn.drop(columns=drop_nn, axis=1)

num_features_nn = len(test_nn.columns)

In [31]:
# スタッキング
def predict_cv(model, X, y, df_test):
    preds = []
    preds_test = []
    va_idxes = []

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for i, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
        tr_x, va_x = X.iloc[tr_idx], X.iloc[va_idx]
        tr_y, va_y = y.iloc[tr_idx], y.iloc[va_idx]
        
        # Target Encoding
        if 'AgeGroup' in tr_x.columns:
            tmp = tr_x.groupby(by=['AgeGroup', 'ProductPitched'], as_index=False)['ProdTaken'].mean()
            tmp = tmp.rename(columns={'ProdTaken': 'ContractRate_G4'})
            tr_x = tr_x.merge(tmp, on=['AgeGroup', 'ProductPitched'], how='left')
            va_x = va_x.merge(tmp, on=['AgeGroup', 'ProductPitched'], how='left')
            
            tr_x = tr_x.drop(labels='ProdTaken', axis=1)
            va_x = va_x.drop(labels='ProdTaken', axis=1)
        
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(df_test)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [32]:
# 1層目のモデル
models_gbdt = [
    Model1_CatBoost_1(),
    Model1_CatBoost_2(),
    Model1_CatBoost_3(),
    #Model1_XGBoost_1(),
    #Model1_XGBoost_2(),
    #Model1_XGBoost_3(),
    #Model1_LightGBM_1(),
    #Model1_LightGBM_2(),
    #Model1_LightGBM_3()
]

models_nn = [
    #Model1_RandomForest_1(),
    Model1_RandomForest_2(),
    #Model1_RandomForest_3()
    Model1_NN_1(input_shape=num_features_nn),
    #Model1_NN_2(input_shape=num_features_nn),
    Model1_NN_all_1(input_shape=num_features_nn),
    #Model1_NN_all_2(input_shape=num_features_nn),
    #Model1_TabNet_1(input_dim=num_features_nn),
    #Model1_TabNet_2(input_dim=num_features_nn),
    Model1_Logistic_1(),
    #Model1_Logistic_2(),
    #Model1_Logistic_3(),
    #Model1_Logistic_4()
]

In [34]:
pred_train_list = []
pred_test_list = []

for model in models_gbdt:
    pred_train, pred_test = predict_cv(model, X_gb, y_gb, test_gb)
    pred_train_list.append(pred_train)
    pred_test_list.append(pred_test)

for model in models_nn:
    pred_train, pred_test = predict_cv(model, X_nn, y_nn, test_nn)
    pred_train_list.append(pred_train)
    pred_test_list.append(pred_test)

0:	test: 0.7642917	best: 0.7642917 (0)	total: 2.65ms	remaining: 15.9ms
6:	test: 0.8424167	best: 0.8480833 (2)	total: 13.5ms	remaining: 0us

bestTest = 0.8480833333
bestIteration = 2

Shrink model to first 3 iterations.
0:	test: 0.7821429	best: 0.7821429 (0)	total: 2.9ms	remaining: 17.4ms
6:	test: 0.7775630	best: 0.7821429 (0)	total: 19.2ms	remaining: 0us

bestTest = 0.7821428571
bestIteration = 0

Shrink model to first 1 iterations.
0:	test: 0.7328704	best: 0.7328704 (0)	total: 2.88ms	remaining: 17.3ms
6:	test: 0.7538721	best: 0.7672559 (2)	total: 17.2ms	remaining: 0us

bestTest = 0.7672558923
bestIteration = 2

Shrink model to first 3 iterations.
0:	test: 0.7925084	best: 0.7925084 (0)	total: 2.33ms	remaining: 14ms
6:	test: 0.8277778	best: 0.8277778 (6)	total: 15.8ms	remaining: 0us

bestTest = 0.8277777778
bestIteration = 6

0:	test: 0.7035354	best: 0.7035354 (0)	total: 5.33ms	remaining: 32ms
6:	test: 0.7456229	best: 0.7476852 (5)	total: 35.9ms	remaining: 0us

bestTest = 0.7476851852
b

In [35]:
# 1層目のモデルの評価
for i, pred_train in enumerate(pred_train_list):
    auc_score = roc_auc_score(y, pred_train)
    print(f'AUC for model {i+1}: {auc_score}')

AUC for model 1: 0.7679398595244253
AUC for model 2: 0.7895520031709455
AUC for model 3: 0.7981864475668703
AUC for model 4: 0.815336089995734
AUC for model 5: 0.7836803794385679
AUC for model 6: 0.8085709583040817
AUC for model 7: 0.7826961770623744


In [36]:
# 特徴量として使用する列名を作成
column_names = [f'pred_{i+1}' for i in range(len(pred_train_list))]

# 予測値を特徴量としてデータフレームを作成
train_x_2 = pd.DataFrame(
    {f'pred_{i+1}': pred_train_list[i] for i in range(len(pred_train_list))},
    columns=column_names
)

test_x_2 = pd.DataFrame(
    {f'pred_{i+1}': pred_test_list[i] for i in range(len(pred_test_list))},
    columns=column_names
)

In [37]:
# 2層目のモデル
# pred_train_2は、2層目のモデルの学習データのクロスバリデーションでの予測値
# pred_test_2は、2層目のモデルのテストデータの予測値
model_2 = Model2_Logistic()
pred_train_2, pred_test_2 = predict_cv(model_2, train_x_2, y, test_x_2)
print(f'AUC: {roc_auc_score(y, pred_train_2)}')

AUC: 0.8087758605052686


In [ ]:
index = pd.read_csv('data/test.csv')['id'].values
df_submit = pd.DataFrame({
    "id": index,
    "prediction": pred_test_2
})

In [ ]:
path = 'submission/submit_27.csv'

In [ ]:
df_submit.to_csv(path, index=False, header=None)